In [2]:
# Will be used os so we can use nice python 
# syntax to mess with our filesystem instead of 
# ugly bash
import os, shutil
if not os.path.exists('./cancerData'):
    os.makedirs('./cancerData');

In [3]:
# Now we will go in a python for loop and create 
# some subfolders in the cancerData folder.
# We will have a subfolder per patient. For the
# time being, we assume we have 65 patients.
# in reality we have less, but that will just give
# us some empty folders in the end
# here is a bash variable
!incrementer=1;
for i in range(1,66):
    if not os.path.exists('./cancerData/bc'+str(i)):
        os.makedirs('./cancerData/bc'+str(i));

In [7]:
# The following script will go through the chemo folder and 
# find all the patients. For each patient, it will 
# look for folders of the form v1-v4 or Visit1-Visit4 or V1-V4 
# etc, if it finds such a folder, it will create a corresponding
# folder in the cancerData/bc(patient#)/ folder
# The filepath to the chemodata must be input to walk
chemoPath = '/mnt/e/Data Sharing_BreastChemo/Breast_Chemo_Study';
subFolders = next(os.walk(chemoPath))[1];
# SubFolders holds the list of subfolders of chemoPath
print(subFolders);
# Next we traverse the subfolders and visit any subfolder 
# beginning with BreastChemo, all the chemo data should
# be in these subfolders.
for folder in subFolders:
    if folder[0:11] == 'BreastChemo':
        # os.walk returns a list with the first entry
        # the directory, the second entry ([1]) is a list
        # of subfolders, and the third entry ([2]) is a 
        # list of subfiles.
        visits = next(os.walk(chemoPath+'/'+folder));
        print(visits[1])
        # for each folder called Visit1, Visit2, Visit3, Visit4
        # V1, V2, V3, V4, or visit1, visit2, etc, we will make 
        # the appropriate folder in our directory
        for subsubFolder in visits[1]:
            # There is a stupid situation to handle where instead of
            # going into the visits folder we have to go into BCiVj 
            # folders to get slice data.
            visits2 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder));
            for subsubFolder2 in visits2[1]:
                # The normal workflow is just to go into visits folders
                if subsubFolder2[:5] == 'Visit' or subsubFolder2[-2] == 'V' or subsubFolder2[:5] == 'visit':
                    print(folder[-1]);
                    # For each folder of Chemo data, and for each visit in that 
                    # folder, makes a V folder in our file system to hold relavent
                    # information for the future in our file system.
                    if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]):
                        os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]);
                    # We add a L1 subfolder which will contain the data for lession 1 for
                    # the patient. Some patients will also have an L2 subfolder if they have a second lession
                    if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'/L1'):
                        os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'/L1');

                
                    # Next, we must determine if this patient has 1 or 2 tumors. This is a tricky step 
                    # because there was no standard way of displaying the number of lessions, some patients
                    # have the lessions data indicated in one way and others in another. In each visit, we 
                    # will have a subfolder for each Lession labeled L1 and L2, even patients with 1 lession
                    # will have an L1 folder. This is to stay consistent with our file structure.
                    try :
                        data = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder2));
                    except :
                        print("No data");
                        break;
                    for sub3Folder in data[1]:
                        # Often the substring L2 can be found in a file label indicating data for Lession 2
                        if 'L2' in sub3Folder:
                            print("Lession 2 found");
                            print("Patient "+folder);
                            # Make a L2 subfolder to hold lession 2 data if it does not already exist
                            if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'/L2'):
                                os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'/L2');

                                    

                        # Sometimes however, the substring L2 can only be found in a processing subfolder,
                        # so we have to go another step into the file system before we can find out whether we
                        # have more than 1 lession. And sometimes, it appears the second lession is labeled as
                        # lymph (Patient 2 visit 2)
                        else :
                            # We ignore any processing done with adjusted values
                            if 'processing' in sub3Folder or 'Processing' in sub3Folder and 'adjusted' not in sub3Folder:
                                data2 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder2+'/'+sub3Folder));
                                # We run this loop once only adding pix_by_pix data if final is found,
                                finalFound = 0;
                                for sub4Folder in data2[1]:
                                    if 'L2' in sub4Folder:
                                        print("Lession 2 found");
                                        print("Patient "+folder);
                                        # Make a L2 subfolder to hold lession 2 data if it does not already exist
                                        if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'/L2'):
                                            os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'/L2');
                                    if (('final' in sub4Folder) or ('Final' in sub4Folder)) and ('pix_by_pix' in sub4Folder):
                                        if ('L2' in sub4Folder) and ('pix_by_pix' in sub4Folder):
                                            data3 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder2+'/'+sub3Folder+'/'+sub4Folder));
                                            for sub5Folder in data3[1]:
                                                if 'slice' == sub5Folder[:5] or 'Slice' == sub5Folder[:5]:
                                                    if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'/L2/S'+sub5Folder[5:]):
                                                        os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'/L2/S'+sub5Folder[5:]);
                                                    # Now that we have found and added a slice, let us put the excel sheet from
                                                    # inside the slice into our file system
                                                    data4 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder2+'/'+sub3Folder+'/'+sub4Folder+'/'+sub5Folder));
                                                    # The files should be in the 3rd member of the list
                                                    for subfiles in data4[2]:
                                                        # if pix_by_pix is in the subfiles name, and it is
                                                        # an xls file copy it to our directory
                                                        if 'pix_by_pix' in subfiles and 'xls' in subfiles:
                                                            tempTargetPath = './cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'L2/S'+sub5Folder[5:]+'/'+subfiles;
                                                            tempDataPath = chemoPath+'/'+folder+'/'+subsubFolder2+'/'+sub3Folder+'/'+sub4Folder+'/'+sub5Folder+'/'+subfiles; 
                                                            if not os.path.exists(tempTargetPath):
                                                                shutil.copy(tempDataPath,tempTargetPath);
                                                            # If we are here, we found a pix_by_pix xls sheet inside a
                                                            # pix_by_pix final folder, we update final found to 1
                                                            finalFound = 1;
                                        # The final step is to put the contents of pix_by_pix into the L1 L2 folders 
                                        elif 'pix_by_pix' in sub4Folder:
                                            data3 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder2+'/'+sub3Folder+'/'+sub4Folder));
                                            for sub5Folder in data3[1]:
                                                if 'slice' == sub5Folder[:5] or 'Slice' == sub5Folder[:5]:
                                                    if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'/L1/S'+sub5Folder[5:]):
                                                        os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'/L1/S'+sub5Folder[5:]);
                                                    # Now that we have found and added a slice, let us put the excel sheet from
                                                    # inside the slice into our file system
                                                    data4 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder2+'/'+sub3Folder+'/'+sub4Folder+'/'+sub5Folder));
                                                    # The files should be in the 3rd member of the list
                                                    for subfiles in data4[2]:
                                                        # if pix_by_pix is in the subfiles name, and it is
                                                        # an xls file copy it to our directory
                                                        if 'pix_by_pix' in subfiles and 'xls' in subfiles:
                                                            tempTargetPath = './cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'L1/S'+sub5Folder[5:]+'/'+subfiles;
                                                            tempDataPath = chemoPath+'/'+folder+'/'+subsubFolder2+'/'+sub3Folder+'/'+sub4Folder+'/'+sub5Folder+'/'+subfiles; 
                                                            if not os.path.exists(tempTargetPath):
                                                                shutil.copy(tempDataPath,tempTargetPath);
                                                            # If we are here, we found a pix_by_pix xls sheet inside a
                                                            # pix_by_pix final folder, we update final found to 1
                                                            finalFound = 1;
                                                            
                                # We run this loop only if final and pix_by_pix was not found,
                                if finalFound == 0:
                                    for sub4Folder in data2[1]:
                                        if 'L2' in sub4Folder:
                                            print("Lession 2 found");
                                            print("Patient "+folder);
                                            # Make a L2 subfolder to hold lession 2 data if it does not already exist
                                            if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'/L2'):
                                                os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'/L2');
                                        if ('L2' in sub4Folder) and ('pix_by_pix' in sub4Folder):
                                            data3 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder2+'/'+sub3Folder+'/'+sub4Folder));
                                            for sub5Folder in data3[1]:
                                                if 'slice' == sub5Folder[:5] or 'Slice' == sub5Folder[:5]:
                                                    if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'/L2/S'+sub5Folder[5:]):
                                                        os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'/L2/S'+sub5Folder[5:]);
                                                    # Now that we have found and added a slice, let us put the excel sheet from
                                                    # inside the slice into our file system
                                                    data4 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder2+'/'+sub3Folder+'/'+sub4Folder+'/'+sub5Folder));
                                                    # The files should be in the 3rd member of the list
                                                    for subfiles in data4[2]:
                                                        # if pix_by_pix is in the subfiles name, and it is
                                                        # an xls file copy it to our directory
                                                        if 'pix_by_pix' in subfiles and 'xls' in subfiles:
                                                            tempTargetPath = './cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'L2/S'+sub5Folder[5:]+'/'+subfiles;
                                                            tempDataPath = chemoPath+'/'+folder+'/'+subsubFolder2+'/'+sub3Folder+'/'+sub4Folder+'/'+sub5Folder+'/'+subfiles; 
                                                            if not os.path.exists(tempTargetPath):
                                                                shutil.copy(tempDataPath,tempTargetPath);
                                        # The final step is to put the contents of pix_by_pix into the L1 L2 folders 
                                        elif 'pix_by_pix' in sub4Folder:
                                            data3 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder2+'/'+sub3Folder+'/'+sub4Folder));
                                            for sub5Folder in data3[1]:
                                                if 'slice' == sub5Folder[:5] or 'Slice' == sub5Folder[:5]:
                                                    if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'/L1/S'+sub5Folder[5:]):
                                                        os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'/L1/S'+sub5Folder[5:]);
                                                    # Now that we have found and added a slice, let us put the excel sheet from
                                                    # inside the slice into our file system
                                                    data4 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder2+'/'+sub3Folder+'/'+sub4Folder+'/'+sub5Folder));
                                                    # The files should be in the 3rd member of the list
                                                    for subfiles in data4[2]:
                                                        # if pix_by_pix is in the subfiles name, and it is
                                                        # an xls file copy it to our directory
                                                        if 'pix_by_pix' in subfiles and 'xls' in subfiles:
                                                            tempTargetPath = './cancerData/bc'+folder[11:]+'/V'+subsubFolder2[-1]+'L1/S'+sub5Folder[5:]+'/'+subfiles;
                                                            tempDataPath = chemoPath+'/'+folder+'/'+subsubFolder2+'/'+sub3Folder+'/'+sub4Folder+'/'+sub5Folder+'/'+subfiles; 
                                                            if not os.path.exists(tempTargetPath):
                                                                shutil.copy(tempDataPath,tempTargetPath);
                            
    
            # The normal workflow is just to go into visits folders
            if subsubFolder[:5] == 'Visit' or subsubFolder[-2] == 'V' or subsubFolder[:5] == 'visit':
                print(folder[-1]);
                # For each folder of Chemo data, and for each visit in that 
                # folder, makes a V folder in our file system to hold relavent
                # information for the future in our file system.
                if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]):
                    os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]);
                # We add a L1 subfolder which will contain the data for lession 1 for
                # the patient. Some patients will also have an L2 subfolder if they have a second lession
                if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]+'/L1'):
                    os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]+'/L1');

                
                # Next, we must determine if this patient has 1 or 2 tumors. This is a tricky step 
                # because there was no standard way of displaying the number of lessions, some patients
                # have the lessions data indicated in one way and others in another. In each visit, we 
                # will have a subfolder for each Lession labeled L1 and L2, even patients with 1 lession
                # will have an L1 folder. This is to stay consistent with our file structure.
                data = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder));
                for sub3Folder in data[1]:
                    # Often the substring L2 can be found in a file label indicating data for Lession 2
                    if 'L2' in sub3Folder:
                        print("Lession 2 found");
                        print("Patient "+folder);
                        # Make a L2 subfolder to hold lession 2 data if it does not already exist
                        if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]+'/L2'):
                            os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]+'/L2');

                                    

                    # Sometimes however, the substring L2 can only be found in a processing subfolder,
                    # so we have to go another step into the file system before we can find out whether we
                    # have more than 1 lession. And sometimes, it appears the second lession is labeled as
                    # lymph (Patient 2 visit 2)
                    else :
                        if 'processing' in sub3Folder or 'Processing' in sub3Folder and 'adjusted' not in sub3Folder:
                            data2 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder+'/'+sub3Folder));
                            finalFound = 0;
                            for sub4Folder in data2[1]:
                                if 'L2' in sub4Folder:
                                    print("Lession 2 found");
                                    print("Patient "+folder);
                                    # Make a L2 subfolder to hold lession 2 data if it does not already exist
                                    if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]+'/L2'):
                                        os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]+'/L2');
                                if ('final' in sub4Folder or 'Final' in sub4Folder) and 'pix_by_pix' in sub4Folder:
                                    if ('L2' in sub4Folder) and 'pix_by_pix' in sub4Folder:
                                        data3 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder+'/'+sub3Folder+'/'+sub4Folder));
                                        for sub5Folder in data3[1]:
                                            if 'slice' == sub5Folder[:5] or 'Slice' == sub5Folder[:5]:
                                                if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]+'/L2/S'+sub5Folder[5:]):
                                                    os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]+'/L2/S'+sub5Folder[5:]);
                                                # Now that we have found and added a slice, let us put the excel sheet from
                                                # inside the slice into our file system
                                                data4 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder+'/'+sub3Folder+'/'+sub4Folder+'/'+sub5Folder));
                                                # The files should be in the 3rd member of the list
                                                for subfiles in data4[2]:
                                                    # if pix_by_pix is in the subfiles name, and it is
                                                    # an xls file copy it to our directory
                                                    if 'pix_by_pix' in subfiles and 'xls' in subfiles:
                                                        tempTargetPath = './cancerData/bc'+folder[11:]+'/V'+subsubFolder[-1]+'/L2/S'+sub5Folder[5:]+'/'+subfiles;
                                                        tempDataPath = chemoPath+'/'+folder+'/'+subsubFolder+'/'+sub3Folder+'/'+sub4Folder+'/'+sub5Folder+'/'+subfiles; 
                                                        finalFound = 1;
                                                        if not os.path.exists(tempTargetPath):
                                                            shutil.copy(tempDataPath,tempTargetPath);
                                    # The final step is to put the contents of pix_by_pix into the L1 L2 folders 
                                    elif 'pix_by_pix' in sub4Folder:
                                        data3 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder+'/'+sub3Folder+'/'+sub4Folder));
                                        for sub5Folder in data3[1]:
                                            if 'slice' == sub5Folder[:5] or 'Slice' == sub5Folder[:5]:
                                                if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]+'/L1/S'+sub5Folder[5:]):
                                                    os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]+'/L1/S'+sub5Folder[5:]);
                                                # Now that we have found and added a slice, let us put the excel sheet from
                                                # inside the slice into our file system
                                                data4 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder+'/'+sub3Folder+'/'+sub4Folder+'/'+sub5Folder));
                                                # The files should be in the 3rd member of the list
                                                for subfiles in data4[2]:
                                                    # if pix_by_pix is in the subfiles name, and it is
                                                    # an xls file copy it to our directory
                                                    if 'pix_by_pix' in subfiles and 'xls' in subfiles:
                                                        tempTargetPath = './cancerData/bc'+folder[11:]+'/V'+subsubFolder[-1]+'/L1/S'+sub5Folder[5:]+'/'+subfiles;
                                                        tempDataPath = chemoPath+'/'+folder+'/'+subsubFolder+'/'+sub3Folder+'/'+sub4Folder+'/'+sub5Folder+'/'+subfiles; 
                                                        finalFound = 1;
                                                        if not os.path.exists(tempTargetPath):
                                                            shutil.copy(tempDataPath,tempTargetPath);
                            if finalFound == 0:
                                for sub4Folder in data2[1]:
                                    if 'L2' in sub4Folder:
                                        print("Lession 2 found");
                                        print("Patient "+folder);
                                        # Make a L2 subfolder to hold lession 2 data if it does not already exist
                                        if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]+'/L2'):
                                            os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]+'/L2');
                                    if ('L2' in sub4Folder) and 'pix_by_pix' in sub4Folder:
                                        data3 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder+'/'+sub3Folder+'/'+sub4Folder));
                                        for sub5Folder in data3[1]:
                                            if 'slice' == sub5Folder[:5] or 'Slice' == sub5Folder[:5]:
                                                if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]+'/L2/S'+sub5Folder[5:]):
                                                    os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]+'/L2/S'+sub5Folder[5:]);
                                                # Now that we have found and added a slice, let us put the excel sheet from
                                                # inside the slice into our file system
                                                data4 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder+'/'+sub3Folder+'/'+sub4Folder+'/'+sub5Folder));
                                                # The files should be in the 3rd member of the list
                                                for subfiles in data4[2]:
                                                    # if pix_by_pix is in the subfiles name, and it is
                                                    # an xls file copy it to our directory
                                                    if 'pix_by_pix' in subfiles and 'xls' in subfiles:
                                                        tempTargetPath = './cancerData/bc'+folder[11:]+'/V'+subsubFolder[-1]+'/L2/S'+sub5Folder[5:]+'/'+subfiles;
                                                        tempDataPath = chemoPath+'/'+folder+'/'+subsubFolder+'/'+sub3Folder+'/'+sub4Folder+'/'+sub5Folder+'/'+subfiles; 
                                                        finalFound = 1;
                                                        if not os.path.exists(tempTargetPath):
                                                            shutil.copy(tempDataPath,tempTargetPath);
                                    # The final step is to put the contents of pix_by_pix into the L1 L2 folders 
                                    elif 'pix_by_pix' in sub4Folder:
                                        data3 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder+'/'+sub3Folder+'/'+sub4Folder));
                                        for sub5Folder in data3[1]:
                                            if 'slice' == sub5Folder[:5] or 'Slice' == sub5Folder[:5]:
                                                if not os.path.exists('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]+'/L1/S'+sub5Folder[5:]):
                                                    os.makedirs('./cancerData/bc'+folder[11:]+'/V'+subsubFolder[5]+'/L1/S'+sub5Folder[5:]);
                                                # Now that we have found and added a slice, let us put the excel sheet from
                                                # inside the slice into our file system
                                                data4 = next(os.walk(chemoPath+'/'+folder+'/'+subsubFolder+'/'+sub3Folder+'/'+sub4Folder+'/'+sub5Folder));
                                                # The files should be in the 3rd member of the list
                                                for subfiles in data4[2]:
                                                    # if pix_by_pix is in the subfiles name, and it is
                                                    # an xls file copy it to our directory
                                                    if 'pix_by_pix' in subfiles and 'xls' in subfiles:
                                                        tempTargetPath = './cancerData/bc'+folder[11:]+'/V'+subsubFolder[-1]+'/L1/S'+sub5Folder[5:]+'/'+subfiles;
                                                        tempDataPath = chemoPath+'/'+folder+'/'+subsubFolder+'/'+sub3Folder+'/'+sub4Folder+'/'+sub5Folder+'/'+subfiles; 
                                                        finalFound = 1;
                                                        if not os.path.exists(tempTargetPath):
                                                            shutil.copy(tempDataPath,tempTargetPath);
                                    
                            

['AIF', 'BreastChemo1', 'BreastChemo10', 'BreastChemo11', 'BreastChemo12', 'BreastChemo13', 'BreastChemo14', 'BreastChemo15', 'BreastChemo16', 'BreastChemo17', 'BreastChemo18', 'BreastChemo19', 'BreastChemo2', 'BreastChemo20', 'BreastChemo21', 'BreastChemo22', 'BreastChemo23', 'BreastChemo24', 'BreastChemo25', 'BreastChemo26', 'BreastChemo27', 'BreastChemo28', 'BreastChemo29', 'BreastChemo3', 'BreastChemo30', 'BreastChemo31', 'BreastChemo32', 'BreastChemo33', 'BreastChemo34', 'BreastChemo36', 'BreastChemo37', 'BreastChemo38', 'BreastChemo39', 'BreastChemo4', 'BreastChemo40', 'BreastChemo41', 'BreastChemo42', 'BreastChemo43', 'BreastChemo44', 'BreastChemo45', 'BreastChemo46', 'BreastChemo48', 'BreastChemo49', 'BreastChemo5', 'BreastChemo50', 'BreastChemo51', 'BreastChemo52', 'BreastChemo53', 'BreastChemo54', 'BreastChemo55', 'BreastChemo56', 'BreastChemo57', 'BreastChemo58', 'BreastChemo59', 'BreastChemo6', 'BreastChemo60', 'BreastChemo7', 'BreastChemo8', 'BreastChemo9', 'CD31 Staining'

In [ ]:
# Here we will compute the volumes and averages
# for each patient for each tumor, These values will be
# placed in a csv file inside V1, V2, V3, and V4 

In [1]:
# This code handles the actual model
import sklearn as sk
from sklearn import svm
import numpy as np
import pandas as pd
import copy as copy
import scipy
# Grab the data from the file 'TrainCancerData - Sheet1.csv'
train = pd.read_csv('TrainCancerData - Sheet1.csv');
# Grabs the test data from the file 'TestCancerData - Sheet1.csv'
test = pd.read_csv('TestCancerData - Sheet1.csv');
# LOOCV is the Leave One Out cross validation dataset
LOOCV = pd.read_csv('alldata.csv');
# This will hold the label data
ytrain = np.array(train['RCB-class']);
yLOOCV = np.array(LOOCV['RCB-class']);
# The arrays above will hold values of 0,1,2, and 3
# which are the various scores for the tumors, we will
# be looking at a binary classification 0 versus greater
# than 0. (PCR vs non PCR)
for i in range(0,np.shape(ytrain)[0]):
    if ytrain[i]>0:
        ytrain[i] = 1
for i in range(0,np.shape(yLOOCV)[0]):
    if yLOOCV[i] > 0:
        yLOOCV[i] = 1
# This is the test label vector
ytest = np.array(test['RCB-class']);
for i in range(0,np.shape(ytest)[0]):
    if ytest[i] > 0:
        ytest[i] = 1
# We will use a linear support vector machine as our
# model
myModel = svm.SVC(kernel='linear');

In [2]:
# This creates the training and test sets where the features 
# we want to use in our model are passed in as a list
# This code is to be run when we just want to set asside 
# a training and test set
def createTrainTestMatrix(features):
    xtrain = np.zeros((len(ytrain),len(features)));
    xtest = np.zeros((len(ytest),len(features)));
    for i in range(0,len(features)):
        xtrain[:,i] = train[features[i]];
        xtest[:,i] = test[features[i]];
    # scale subtracts out medians and scales by mads
    # why subtract out medians and scale by mads? because
    # for our small dataset, subtracting out means and 
    # scaling by standard deviations is unreliable
    return scale(features,xtrain,xtest)

In [3]:
# Scales and centers the training and test set by using
# the median and mad of the training set.
def scale(features,train,test):
    ctrain = copy.copy(train);
    ctest = copy.copy(test);
    for i in range(0,len(features)):
        median = np.median(ctrain[:,i]);
        # The following snippet, handles the "special" case
        # where the feature has FX in its name, here, any values
        # that are zero are fixed to zero. Im not sure why I do 
        # this. 0's in FX data have special meaning. (no cancer 
        # data)
        if features[i].find('FX') > -1:
            for j in range(0,np.shape(ctrain)[0]):
                if ctrain[j,i] == 0:
                    ctrain[j,i] = median;
            for j in range(0,np.shape(ctest)[0]):
                if ctest[j,i] == 0:
                    ctest[j,i] = median;
        ctrain[:,i] -= median;
        ctest[:,i] -= median;
        mad =sk.metrics.median_absolute_error(ctrain[:,i],np.zeros(np.shape(ctrain)[0]));
        if mad != 0:
            ctrain[:,i] = ctrain[:,i]/(mad+.0);
            ctest[:,i] = ctest[:,i]/(mad+.0);
    return [ctrain,ctest];

In [4]:
# This function creates the training set matrix for
# cross validation, it will work pretty mush the same
# as createTrainTestMatrix but we do not scale because
# for cross validation, we will be scaling inside the 
# validation loop.
def createCVMatrix(features):
    x = np.zeros((len(yLOOCV),len(features)));
    # We do not center and scale this set because that
    # is done inside the cv loop
    for i in  range(0,len(features)):
        x[:,i] = LOOCV[features[i]];
    return x;

In [5]:
# Shuffles the x and y together (x is the training set,
# and y is the label set). This is a very inefficient
# implementation
def shuffle(x,y):
    newX = np.zeros(np.shape(x));
    newY = np.zeros(np.shape(y));
    indices = np.array(range(0,np.shape(x)[0]));
    np.random.shuffle(indices);
    for i in range(0,len(indices)):
        newX[i,:] = x[indices[i]];
        newY[i] = y[indices[i]];
    return [newX,newY];

In [6]:
# This runs a linear support vector machine for the
# situation where we have a training set and a test
# set set asside
def runLinSVM(features,c=None,xtrain=None,xtest=None):
    # Sets the model regularization constant if one 
    # was passed in, otherwise the constant is whatever 
    # the model myModel has the constant set to
    if c != None:
        myModel.C = c;
    # Creates the training and test data matrices
    # based on the selected features, if the training
    # and test set where not passed in
    if xtrain == None or xtest == None:
        [xtrain,xtest] = createTrainTestMatrix(features);
    myModel.fit(xtrain,ytrain);
    # we return xtest and ytest so the user can use these
    # sets to validate their model. myModel is a global variable
    # so it does not need to be returned.
    return [xtest,ytest];

In [7]:
# This function will be needed for leave one out 
# cross validation. It rotates the datasets x and y in
# parallel. The last datapoint of x is sent to the beginning
# the last label of y is sent to the beginning, the second to
# last datapoint in x is sent the the last point, etc
def rotate(x,y):
    xtemp = np.zeros(np.shape(x));
    ytemp = np.zeros(np.shape(y)[0]);
    xtemp[:-1,:] = x[1:,:];
    xtemp[-1,:] = x[0,:];
    ytemp[:-1] = y[1:];
    ytemp[-1] = y[0];
    return [xtemp,ytemp];

In [8]:
# This is the main code for the cross validation loop
# n is the number of values of the regularization constant
# to try, c is the smallest regularization constant to try,
# so we will try c, c*2^1, c*2^2, ... c*2^n as our regularization
# constants. It is widely known that regularization constants 
# should be searched for on an exponential scale which is why we
# use 2^i, x is the optional data matrix, if no x is passed in
# x is created from the passed in features
def runCVSVM2(features,n=5,c=None,x=None):
    # will contain the ideal c values from the cross validation loops
    myCs = [];
    # If there was no specified training set (x),
    # we create it here. We have no test set as we
    # are using leave one out cross validation
    if x == None:
        x = createCVMatrix(features)
    y = np.array(yLOOCV);
    # This is where all the magic happens. Notice the quadruple
    # nested loop. Wow, so inneficient. 
    xSize = np.shape(x);
    # Our outer loop (test) predictions will be stored in fpreds
    fpreds = np.zeros(xSize[0]);
    # Our outer loop (test) svm outputs will be stored here
    fvals = np.zeros(xSize[0]);
    # Outer loop
    for i in range(0,xSize[0]):
        # Our training and validation set will be
        # x[1:,:] (all of x except the first datapoint)
        xTrain = copy.copy(x[1:,:]);
        # Our training and validation labels will be
        # y[1:] (all of y except the first label)
        yTrain = copy.copy(y[1:]);
        # This array will hold the best auc scores for our
        # inner validation loop
        bestAuc = np.zeros(xSize[0]-1);
        for j in range(0,xSize[0]-1):
            # Our inner training set is all but the first row
            # of the set we defined in the outer loop 
            xTrain2 = copy.copy(xTrain[1:,:])
            yTrain2 = copy.copy(yTrain[1:])
            # This will hold the auc scores of each run for each 
            # value of the regularization constant
            auc = np.zeros(n);
            for k in range(0,n):
                # Set the regularization constant
                myModel.C = c*2**k;
                # make an array to hold predictions and values
                # or that regularization constant
                preds = np.zeros(xSize[0]-2);
                vals = np.zeros(xSize[0]-2);
                # Now we go through the training set, rotating
                # one guy out and running the model a total of 
                # xSize - 2 times, we have to rescale every time
                # because the scaling needs to be done inside the
                # loop
                for m in range(0,xSize[0]-2):
                    [xTrain3, xTest3] = scale(features,xTrain2[1:,:],np.array([xTrain2[0,:]]));
                    myModel.fit(xTrain3,yTrain2[1:]);
                    preds[m] = myModel.predict(xTest3);
                    vals[m] = np.dot(myModel.coef_,xTest3.T) + myModel.intercept_;
                    [xTrain2,yTrain2] = rotate(xTrain2,yTrain2);
                auc[k] = getAuc(vals,yTrain[1:]);
            bestAuc[j] = np.argmax(auc);
            [xTrain,yTrain] = rotate(xTrain,yTrain);
        # Our final auc will be the median of the best auc,
        # for our small set, we observed this to give the most
        # consistent results
        finalAuc = np.median(bestAuc);
        # The following code is designed to test the effectiveness
        # of this cross validation proceedure. In practice, to 
        # create a model, a practitioner would only need to do
        # a leave one out cross validation proceedure to select the 
        # regularization hyper parameter, and the practitioner would
        # select the regularization parameter that is the median of 
        # the best regularization parameters chosen by the cross 
        # validation proceedure. But we must also evaluate this technique
        # of hyper parameter selection. So we need to run the model on
        # 3 levels of leave one out cross validation.
        myModel.C = c*2**finalAuc;
        myCs.append(c*2**finalAuc);
        [xTrainTemp,xTestTemp] = scale(features,x[1:,:],np.array([x[0,:]]));
        myModel.fit(xTrainTemp,y[1:]);
        fpreds[i] = myModel.predict(xTestTemp);
        fvals[i] = np.dot(myModel.coef_,xTestTemp.T)+myModel.intercept_;
        [x,y] = rotate(x,y);
    print(myCs);
    return [getAccuracy(fpreds,y),getAuc(fvals,y),fvals];

In [9]:
def getAccuracy(h,y):
    return (np.sum(h*y)+np.sum((1-h)*1-y))/(len(y)+.0);

def getAuc(h,y):
    return sk.metrics.roc_auc_score(y,h);

def evalOnTest(features,xtest,ytest):
    myPreds = myModel.predict(xtest);
    sumCorrect = 0;
    for i in range(0,len(ytest)):
        if myPreds[i] == ytest[i]:
            sumCorrect += 1;
    accuracy = (sumCorrect + .0)/len(ytest);
    myPred = np.dot(myModel.coef_,xtest.T).T;
    auroc = sk.metrics.roc_auc_score(ytest,myPred);
    return accuracy,auroc;

In [10]:
# Here are some common combinations of features to 
# run the tests on
# clinical data only
noImageFeatures = ['Age','tumor type','tumor grade','ER','PR','HER2','T','N','M','HR'];
# FXL Visit 4 data, we use the tag 'missing V4' to indicate that there is
# no Visit 4 data for this particular patient which may be useful in the 
# classification problem
FXLV4Features = ['FXL-Ktrans V4','FXL-Ve V4','FXL-kep V4','missing V4'];
# FXR2 Visit 4 data features
FXR2V4Features = ['FXR2-Ktrans V4','FXR2-Ve V4','FXR2-kep V4','missing V4'];
# FXR3 Visit 4 data features
FXR3V4Features = ['FXR3-Ktrans V4','FXR3-Ve V4','FXR3-kep V4','FXR3-Tau V4','missing V4'];
# FXL Visit 3 data, we use the tag 'missing V3' to indicate that there is
# no Visit 3 data for this particular patient which may be useful in the 
# classification problem
FXLV3Features = ['FXL-Ktrans V3','FXL-Ve V3','FXL-kep V3','missing V3'];
# FXR2 Visit 3 data features
FXR2V3Features = ['FXR2-Ktrans V3','FXR2-Ve V3','FXR2-kep V3','missing V3'];
# FXR3 Visit 3 data features
FXR3V3Features = ['FXR3-Ktrans V3','FXR3-Ve V3','FXR3-kep V3','FXR3-Tau V3','missing V3'];
# FXL Visit 2 data, we use the tag 'missing V2' to indicate that there is
# no Visit 2 data for this particular patient which may be useful in the 
# classification problem
FXLV2Features = ['FXL-Ktrans V2','FXL-Ve V2','FXL-kep V2','missing V2'];
# FXR2 Visit 2 data features
FXR2V2Features = ['FXR2-Ktrans V2','FXR2-Ve V2','FXR2-kep V2','missing V2'];
# FXR3 Visit 2 data features
FXR3V2Features = ['FXR3-Ktrans V2','FXR3-Ve V2','FXR3-kep V2','FXR3-Tau V2','missing V2'];
# FXL Visit 1 data, we use the tag 'missing V2' to indicate that there is
# no Visit 1 data for this particular patient which may be useful in the 
# classification problem
FXLV1Features = ['FXL-Ktrans V1','FXL-Ve V1','FXL-kep V1','missing V1'];
# FXR2 Visit 1 data features
FXR2V1Features = ['FXR2-Ktrans V1','FXR2-Ve V1','FXR2-kep V1','missing V1'];
# FXR3 Visit 1 data features
FXR3V1Features = ['FXR3-Ktrans V1','FXR3-Ve V1','FXR3-kep V1','FXR3-Tau V1','missing V1'];
# The following feature are going to be the ones we end up using in our tests
FXLV1 = FXLV1Features+noImageFeatures;
FXR2V1 = FXR2V1Features+noImageFeatures;
FXR3V1 = FXR2V1Features+noImageFeatures;
FXLV1V2 = FXLV1 + FXLV2Features;
FXR2V1V2 = FXR2V1 + FXR2V2Features;
FXR3V1V2 = FXR3V1 + FXR3V2Features;
FXLV1V2V3 = FXLV1V2 + FXLV3Features;
FXR2V1V2V3 = FXR2V1V2 + FXR2V3Features;
FXR3V1V2V3 = FXR3V1V2 + FXR3V3Features;
FXLV1V2V3V4 = FXLV1V2V3 + FXLV4Features;
FXR2V1V2V3V4 = FXR2V1V2V3 + FXR2V4Features;
FXR3V1V2V3V4 = FXR3V1V2V3 + FXR3V4Features;

In [11]:
FXR2V1V2V3V4

['FXR2-Ktrans V1',
 'FXR2-Ve V1',
 'FXR2-kep V1',
 'missing V1',
 'Age',
 'tumor type',
 'tumor grade',
 'ER',
 'PR',
 'HER2',
 'T',
 'N',
 'M',
 'HR',
 'FXR2-Ktrans V2',
 'FXR2-Ve V2',
 'FXR2-kep V2',
 'missing V2',
 'FXR2-Ktrans V3',
 'FXR2-Ve V3',
 'FXR2-kep V3',
 'missing V3',
 'FXR2-Ktrans V4',
 'FXR2-Ve V4',
 'FXR2-kep V4',
 'missing V4']

In [ ]:
[acc,auc,ranks] = runCVSVM2(FXR3V1V2V3V4,n=1,c=2**0);

In [39]:
auc

0.572072072072072

In [38]:
acc


0.0